### Analysis of Nashville housing dataset

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import re

In [ ]:
#import csv
df = pd.read_csv("/home/mali/datacleaning_validation/nashvillehousing(txt)/Nashville Housing Data for Data Cleaning (reuploaded).csv", delimiter= ";")

In [ ]:
#data types, memory usage, and data range
df.info()

In [ ]:
#removing missing/null values from dataframe
df = df.dropna()

In [ ]:
#changing datatypes
#downcast to reduce memory usage of the data in the columns

def inter(column):
    pattern = r"[^0-9]" #pattern for excluding numbers
    column = column.replace(pattern, "", regex=True)
    return pd.to_numeric(column, downcast="integer").convert_dtypes()

def date_format(column):
    column = pd.to_datetime(column)
    column = column.dt.strftime("%d-%m-%Y")
    return column

def year_format(column):
    column = pd.to_numeric(column, downcast="integer").convert_dtypes()
    column = pd.to_datetime(column, format = "%Y")
    column = column.dt.strftime("%Y")
    return column

def to_bool(column):
    column = column.replace({"No":"False", "N":"False"})
    column = column.replace({"Yes":"True", "Y":"True"})
    return column
def to_float(column):
    column = column.str.replace(",",".")
    column = pd.to_numeric(column, downcast="float")
    return column


In [ ]:
df["SalePrice"] = inter(df["SalePrice"]) #Int
df["LandValue"] = inter(df["LandValue"]) #Int
df["BuildingValue"] = inter(df["BuildingValue"]) #Int
df["TotalValue"] = inter(df["TotalValue"]) #Int
df["Acreage"] = to_float(df["Acreage"]) #float

df["Bedrooms"] = inter(df["Bedrooms"]) #Int
df["FullBath"] = inter(df["FullBath"]) #Int
df["HalfBath"] = inter(df["HalfBath"]) #Int

df["SaleDate"] = date_format(df["SaleDate"])#obj
df["YearBuilt"] = year_format(df["YearBuilt"])#obj

df["SoldAsVacant"] = to_bool(df["SoldAsVacant"])#Obj


### Data transformation

In [ ]:
#checking the catagorical values to see if there are unexpected values
print(f"{df["TaxDistrict"].unique()}\n\n{df["LandUse"].unique()}\n\n{df["TaxDistrict"].unique()} ")

In [ ]:
#replacing GREENBELT/RES\r\nGRRENBELT/RES as it seems to be a duplicate of GREENBELT/RES
df["LandUse"] = df["LandUse"].replace({"GREENBELT/RES\r\nGRRENBELT/RES":"GREENBELT/RES"})

#replacing "&" with "," in "OwnerName" // replacing "." with " " in "OwnerName"
df["OwnerName"] = df["OwnerName"].str.replace("&",",").str.replace(".","")

In [ ]:
#split "PropertyAddress" into "streetAddress" and "City"
df[["StreetAddress", "City"]] = df["PropertyAddress"].str.split(", ", expand=True)

#split "PropertyAddress" into "streetAddress" and "City"
df[["OwnerStreetAddress", "OwnerCity","OwnerState"]] = df["OwnerAddress"].str.split(", ", expand=True)

#drop old columns
#drop OwnerState as 100% were from TN
df = df.drop(columns=["PropertyAddress","OwnerAddress","OwnerState"])



#### Outliers

##### Based on the prelim analysis it was determined for there to be outliers in these columns "SalePrice", "Acreage", "landValue", "BuildingValue", "TotalValue", "Bedrooms", "FullBath"

Most of these may be connected by certain entries in the dataset



In [ ]:
def outlier(df,column):
    # calculate IQR for column Height
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # identify outliers
    threshold = 1.5
    outliers = df[(df[column] < Q1 - threshold * IQR) | (df[column] > Q3 + threshold * IQR)]
    # drop rows containing outliers
    df = df.drop(outliers.index)
    return df


In [ ]:
df = outlier(df,"LandValue")
df = outlier(df,"BuildingValue")
df = outlier(df,"TotalValue")

df = outlier(df,"SalePrice")
df = outlier(df,"Acreage")

df = outlier(df,"Bedrooms")
df = outlier(df,"FullBath")

#### Validation of data

In [ ]:
df.describe()

#### Conclusion drawn in results.md